# Taxi  Data 


This part of the report refers to the analysis of the taxi data. These data include the complete catalog of the TLC Trip record  data for 2009 through 2015. 

## Libraries
In the cell bellow, the libraries that will be used for the manipulation of the data are imported.
* Pandas is a software library written for data manipulation and analysis. 
* Numpy is a software library that is used for scientific computing in python. 
* Sqlalchemy is an opensource SQL toolkit that is used in python.
* BeatifulSoup is a python library for pulling data out of HTML and XML files.



In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm
from sqlalchemy import create_engine



## Web scraping the yellow taxi data 

In the cell bellow comments are given in between the code to explain the web scraping of csv urls as well as the concatenation of the final dataframe. 

Because of the large size of the data the library dask was installed, allowing for parallel computing, enabling data manipulation and computations of large datasets where Panda fails.




In [2]:
from dask import dataframe as df
# Web scraping the yellow taxi data, from the website using BeautifulSoup 
request = requests.get("https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page")
response = BeautifulSoup(request.content, "lxml")
table_csv = response.find("div",{"class":"faq-v1"})
# Getting the csv url, month and year for downloading the data programmatically later
months = table_csv.find_all("a",{"title":"Yellow Taxi Trip Records"})
hrefs = [i.get("href") for i in months]
month = [int(hrefs[i][-6:-4]) for i in range(len(hrefs))]
year = [int(hrefs[i][-11:-7]) for i in range(len(hrefs))]
# Created a list of tuples containing url paths, and their corresponding month + year
csv_data_tuple = list(zip(year,month,hrefs))
# filter the list of tuples for data between 2009 January and 2015 June
csv_data = [i for i in csv_data_tuple if (i[0]>=2009)&(i[0]<=2015)&((i[0]<2015 )|(i[1]<7))]
# Empty data frame to be use as master dataframe
df_taxi = pd.DataFrame()
i = 0

# Defining various column name changes throughout the data and mapping them to a common name 
columns1 =["tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","fare_amount","tip_amount"]
columns3 = [" pickup_datetime", " dropoff_datetime", " passenger_count"," trip_distance"," pickup_longitude", " pickup_latitude", " dropoff_longitude"," dropoff_latitude", " fare_amount", " tip_amount"]
columns2= ["Trip_Pickup_DateTime","Trip_Dropoff_DateTime","Passenger_Count","Trip_Distance","Start_Lon","Start_Lat","End_Lon","End_Lat","Fare_Amt","Tip_Amt"]
columns4 = ["pickup_datetime", "dropoff_datetime", "passenger_count","trip_distance","pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "fare_amount", "tip_amount"]
mapping2 = {columns2[i]:columns1[i] for i in range(len(columns1))}
mapping3 = {columns3[i]:columns1[i] for i in range(len(columns1))}
mapping4 = {columns4[i]:columns1[i] for i in range(len(columns1))}

# Looping throught the url list, downloading each csv using dask dataframe, filtering to desired columns
# appneding to the master dataframe
for year,month,url in csv_data:
    # reading sampled 3000 rows of data as dask dataframe to reduce memory usage
    dask_df = df.read_csv(url, assume_missing=True, dtype={'store_and_fwd_flag': 'object'}, on_bad_lines="skip").sample(frac=1, random_state=123).head(n=3000)
    if dask_df.columns[1]=="tpep_pickup_datetime":
        
        df_final = dask_df[["tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","fare_amount","tip_amount"]]
        df_taxi = pd.concat([df_taxi,df_final.dropna()])    
    elif dask_df.columns[1]=="Trip_Pickup_DateTime":
        
        dask_df = dask_df.rename(columns=mapping2)
        df_final = dask_df[["tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","fare_amount","tip_amount"]]
        df_taxi = pd.concat([df_taxi,df_final.dropna()])
    elif dask_df.columns[1]==" pickup_datetime":
        
        dask_df = dask_df.rename(columns=mapping3)
        df_final = dask_df[["tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","fare_amount","tip_amount"]]
        df_taxi = pd.concat([df_taxi,df_final.dropna()])
    elif dask_df.columns[1]=="pickup_datetime":
        
        dask_df = dask_df.rename(columns=mapping4)
        df_final = dask_df[["tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","fare_amount","tip_amount"]]
        df_taxi = pd.concat([df_taxi,df_final.dropna()])


/Applications/Python/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Applications/Python/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/Applications/Python/anaconda3/lib/python3.8/site-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


## Filtering for longitude and latitude 

The cell bellow allows the filtering of the data by removing from the dataset the data that are not within the pre-specified longitude and latitude coordinates

In [3]:
df_taxi_filtered = df_taxi[(df_taxi["pickup_longitude"]< -73.717047)&(df_taxi["pickup_longitude"]> -74.242330)&
        (df_taxi["dropoff_longitude"]< -73.717047)&(df_taxi["dropoff_longitude"]> -74.242330)&
       (df_taxi["pickup_latitude"]< 40.908524 )&(df_taxi["pickup_latitude"]> 40.560445)&
        (df_taxi["dropoff_latitude"]< 40.908524)&(df_taxi["dropoff_latitude"]> 40.560445)].copy()
df_taxi_filtered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount
24633,2015-01-17 02:06:49,2015-01-17 02:14:14,1.0,1.60,-73.989265,40.758244,-73.966850,40.756851,7.5,1.75
153097,2015-01-03 22:03:51,2015-01-03 22:20:34,1.0,4.20,-73.989616,40.762238,-73.966606,40.801800,16.5,0.00
108350,2015-01-25 00:15:21,2015-01-25 00:24:21,1.0,1.96,-73.947731,40.711311,-73.952950,40.729382,8.5,1.50
103754,2015-01-24 23:54:49,2015-01-25 00:06:01,3.0,4.05,-73.983009,40.766602,-73.938210,40.798908,13.0,0.00
338912,2015-01-13 19:08:42,2015-01-13 19:21:58,1.0,1.70,-73.989365,40.773693,-73.976028,40.758614,10.0,1.50
...,...,...,...,...,...,...,...,...,...,...
182287,2009-12-06 20:16:06,2009-12-06 20:33:34,2.0,4.00,-73.956395,40.771544,-73.994272,40.722963,12.9,0.00
160876,2009-12-14 22:13:00,2009-12-14 22:19:00,1.0,1.40,-73.991657,40.735145,-74.003693,40.747633,5.7,2.00
266566,2009-12-04 09:00:04,2009-12-04 09:12:07,2.0,1.50,-73.985235,40.748420,-73.983555,40.764730,8.1,0.00
279142,2009-12-27 06:33:00,2009-12-27 06:38:00,1.0,1.61,-73.988680,40.768862,-74.002305,40.752652,6.1,0.00


## Manipulating the date

Now we format the date column for future data manipulation based on date. 

In [4]:
df_taxi_filtered["tpep_pickup_datetime"] = pd.to_datetime(df_taxi_filtered["tpep_pickup_datetime"], infer_datetime_format=True)
df_taxi_filtered["tpep_dropoff_datetime"] = pd.to_datetime(df_taxi_filtered["tpep_dropoff_datetime"], infer_datetime_format=True)

df_taxi_filtered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount
24633,2015-01-17 02:06:49,2015-01-17 02:14:14,1.0,1.60,-73.989265,40.758244,-73.966850,40.756851,7.5,1.75
153097,2015-01-03 22:03:51,2015-01-03 22:20:34,1.0,4.20,-73.989616,40.762238,-73.966606,40.801800,16.5,0.00
108350,2015-01-25 00:15:21,2015-01-25 00:24:21,1.0,1.96,-73.947731,40.711311,-73.952950,40.729382,8.5,1.50
103754,2015-01-24 23:54:49,2015-01-25 00:06:01,3.0,4.05,-73.983009,40.766602,-73.938210,40.798908,13.0,0.00
338912,2015-01-13 19:08:42,2015-01-13 19:21:58,1.0,1.70,-73.989365,40.773693,-73.976028,40.758614,10.0,1.50
...,...,...,...,...,...,...,...,...,...,...
182287,2009-12-06 20:16:06,2009-12-06 20:33:34,2.0,4.00,-73.956395,40.771544,-73.994272,40.722963,12.9,0.00
160876,2009-12-14 22:13:00,2009-12-14 22:19:00,1.0,1.40,-73.991657,40.735145,-74.003693,40.747633,5.7,2.00
266566,2009-12-04 09:00:04,2009-12-04 09:12:07,2.0,1.50,-73.985235,40.748420,-73.983555,40.764730,8.1,0.00
279142,2009-12-27 06:33:00,2009-12-27 06:38:00,1.0,1.61,-73.988680,40.768862,-74.002305,40.752652,6.1,0.00


Here, we manipulate the data by converting the passenger count of each taxi to an integer. 

In [5]:
# passenger count to integer
df_taxi_filtered["passenger_count"] = df_taxi_filtered["passenger_count"].astype("int64")
df_taxi_filtered

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount
24633,2015-01-17 02:06:49,2015-01-17 02:14:14,1,1.60,-73.989265,40.758244,-73.966850,40.756851,7.5,1.75
153097,2015-01-03 22:03:51,2015-01-03 22:20:34,1,4.20,-73.989616,40.762238,-73.966606,40.801800,16.5,0.00
108350,2015-01-25 00:15:21,2015-01-25 00:24:21,1,1.96,-73.947731,40.711311,-73.952950,40.729382,8.5,1.50
103754,2015-01-24 23:54:49,2015-01-25 00:06:01,3,4.05,-73.983009,40.766602,-73.938210,40.798908,13.0,0.00
338912,2015-01-13 19:08:42,2015-01-13 19:21:58,1,1.70,-73.989365,40.773693,-73.976028,40.758614,10.0,1.50
...,...,...,...,...,...,...,...,...,...,...
182287,2009-12-06 20:16:06,2009-12-06 20:33:34,2,4.00,-73.956395,40.771544,-73.994272,40.722963,12.9,0.00
160876,2009-12-14 22:13:00,2009-12-14 22:19:00,1,1.40,-73.991657,40.735145,-74.003693,40.747633,5.7,2.00
266566,2009-12-04 09:00:04,2009-12-04 09:12:07,2,1.50,-73.985235,40.748420,-73.983555,40.764730,8.1,0.00
279142,2009-12-27 06:33:00,2009-12-27 06:38:00,1,1.61,-73.988680,40.768862,-74.002305,40.752652,6.1,0.00


## Data sampling 

Since the taxi data are significantly more than the uber data, the size of the taxi dataset is reduced in the cell bellow by randomly sampling the same number of data that the uber dataset consists of. 

In [6]:
# sample to uber data size i.e. 195472
df_yellow_taxi = df_taxi_filtered.sample(n=195472, random_state=123)
df_yellow_taxi

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,tip_amount
173040,2012-03-29 23:35:00,2012-03-29 23:44:00,1,2.69,-73.983392,40.755975,-73.955603,40.782873,8.5,0.00
237183,2009-12-05 00:29:00,2009-12-05 00:41:00,1,1.56,-74.009293,40.730890,-73.994308,40.746597,7.7,0.00
214190,2012-12-01 02:30:00,2012-12-01 02:35:00,1,1.04,-73.985315,40.732007,-73.981763,40.743142,6.0,1.62
306074,2012-02-22 13:05:52,2012-02-22 13:10:09,2,0.70,-74.000994,40.731393,-74.003211,40.723128,4.5,1.50
135654,2012-04-13 02:53:00,2012-04-13 03:07:00,2,4.82,-73.982425,40.745733,-73.913632,40.744407,13.3,3.45
...,...,...,...,...,...,...,...,...,...,...
323729,2015-01-16 19:36:24,2015-01-16 19:46:10,2,1.13,-74.002724,40.717899,-73.993317,40.719059,8.0,0.00
189103,2011-04-29 15:48:30,2011-04-29 16:07:49,1,3.00,-73.990082,40.746790,-73.960190,40.778994,12.1,0.00
75364,2009-10-12 06:12:00,2009-10-12 06:13:00,3,0.38,-73.979425,40.738957,-73.985657,40.741568,3.3,0.00
110807,2011-09-26 01:16:37,2011-09-26 01:17:58,1,0.50,-73.991484,40.742361,-73.983682,40.739240,3.3,0.00


## Converting Pandas dataframe to SQL Table 

Here, the processed panda dataframe for yellow taxi data is converted into a SQL Table and consequently saved as a table, Yellow_taxi to an SQL Database named project.db

In [13]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////Users/rishabhsalwan/Downloads/Project Data/Project.db', echo=False)
df_yellow_taxi.to_sql('Yellow_Taxi', con=engine)

### Printing the Final Schema
This Function has been used to print the schema of project.db 

In [19]:
def printSchema(connection):
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [20]:
printSchema(engine)

Uber_new:
  0: index(BIGINT)
  1: fare_amount(FLOAT)
  2: pickup_datetime(DATETIME)
  3: pickup_longitude(FLOAT)
  4: pickup_latitude(FLOAT)
  5: dropoff_longitude(FLOAT)
  6: dropoff_latitude(FLOAT)
  7: passenger_count(BIGINT)
  8: trip_distance(FLOAT)
Weather_daily:
  0: index(BIGINT)
  1: DATE(DATETIME)
  2: DailyAverageWindSpeed(FLOAT)
Weather_hourly:
  0: index(BIGINT)
  1: DATE(DATETIME)
  2: LATITUDE(FLOAT)
  3: LONGITUDE(FLOAT)
  4: HourlyPrecipitation(TEXT)
  5: HourlyWindSpeed(FLOAT)
Yellow_Taxi:
  0: index(BIGINT)
  1: tpep_pickup_datetime(DATETIME)
  2: tpep_dropoff_datetime(DATETIME)
  3: passenger_count(BIGINT)
  4: trip_distance(FLOAT)
  5: pickup_longitude(FLOAT)
  6: pickup_latitude(FLOAT)
  7: dropoff_longitude(FLOAT)
  8: dropoff_latitude(FLOAT)
  9: fare_amount(FLOAT)
  10: tip_amount(FLOAT)
